# Generate Origin Destination Cost Matrix
Description: Find and measure the least-cost paths along the network from multiple origins to multiple destinations.
## Setup Environment

In [1]:
import arcpy
from arcpy import env
import os
import pandas as pd

arcpy.CheckOutExtension('Network')
arcpy.CheckOutExtension("Highways")

env.workspace = "C:\data" # directory with your "large roads speeds" and "tracts centroids" subdirectories
env.overwriteOutput = True

centroid_folder = 'tracts centroids' #relative path from env.workspace path
network_dataset_shp = 'large roads speeds\large_roads_speeds.shp' #relative path from env.workspace path
network_dataset_nd = 'large roads speeds\large_roads_speeds_ND.nd' #relative path from env.workspace path

## Setup indexes

In [2]:
def verify_network_dataset_shp_indices(network_dataset_shp):
    '''
    Return True if Network Dataset has the spatial and attribute Indices added
    '''
    #helper functions
    def check_2_fields(fields):
        if len(fields) != 2:
            return False
        for field in fields:
            if len(field) != 1:
                return False
    def extract(fields):
        return [fields[0][0], fields[1][0]]
    indexes = arcpy.ListIndexes(network_dataset_shp)
    fields =  [index.fields for index in indexes]
    def check_fields(spatial_field, attribute_field):
        if spatial_field.name == "Shape" and spatial_field.type == "Geometry":
            if attribute_field.name == "seconds" and attribute_field.type == "Double":
                return True
        return False
    
    #check index fields are the correct format
    if check_2_fields(fields) == False:
        return False
    fields = extract(fields)
    return check_fields(fields[0], fields[1]) or check_fields(fields[1], fields[0])

def verify_centroid_indices(centroid):
    '''
    Return True if centroid has the spatial Index added
    '''
    indexes = arcpy.ListIndexes(centroid)
    if len(indexes) != 1:
        return False
    field = indexes[0].fields[0]
    return field.name == "Shape" and field.type == "Geometry"

In [3]:
def index_setup(network_dataset_shp, centroid):
    if not verify_network_dataset_shp_indices(network_dataset_shp):
        #ND seconds atribute spatial index
        arcpy.AddIndex_management(network_dataset_shp, "seconds")
        #ND spatial index
        arcpy.AddSpatialIndex_management(network_dataset_shp, "")
        assert(verify_network_dataset_shp_indices(network_dataset_shp))
    print("indexes successfully set for network dataset")
    if not verify_centroid_indices(centroid):
        #centroid spatial index
        arcpy.AddSpatialIndex_management(centroid, "")
        assert(verify_centroid_indices(centroid))
    print("indexes successfully set for centroid")

## Setup inputs

In [4]:
def setup_od_inputs(centroid, centroid_directory, network_dataset_nd):
    #Origins setup
    inOrigins = centroid
    inDestinations = inOrigins
    #Network Dataset setup
    inNetworkDataset = network_dataset_nd
    #Output setup
    out_folder_path = centroid_directory
    out_name = "od_output.gdb"
    arcpy.CreateFileGDB_management(out_folder_path, out_name)
    outGeodatabase = os.path.join(out_folder_path, out_name)
    return inOrigins, inDestinations, inNetworkDataset, outGeodatabase

## Generate OD Matrix in geodatabase

In [5]:
def generate_od_matrix(inOrigins, inDestinations, inNetworkDataset, outGeodatabase):
    arcpy.na.GenerateOriginDestinationCostMatrix(inOrigins, inDestinations, inNetworkDataset, outGeodatabase,
                                                  Origin_Destination_Line_Shape='NO_LINES', Save_Output_Network_Analysis_Layer = "NO_SAVE_OUTPUT_LAYER",
                                                Maximum_Snap_Tolerance = 25000)
    #"NO_LINES" boosts performance
    #the output is stored in output.gdb so we don't need the network_analysis_layer.
    print 'completed od matrix successfully'

## Convert OD Matrix From Geodatabase to CSV

In [6]:
def output_csv(centroid_directory, outGeodatabase):
    inTable = os.path.join(outGeodatabase, "ODLines")
    outLocation = centroid_directory
    outTable = "od_output.csv"
    arcpy.TableToTable_conversion(inTable, outLocation, outTable)

## Find Centroid Files

In [7]:
def find_centroid(centroid_folder):
    for directory, _, files in os.walk(os.path.join(env.workspace, centroid_folder)):
        for file in files:
            if ".shp" == file[-4:]:
                centroid_directory = directory
                centroid_shp = os.path.join(directory, file)
                yield centroid_directory, centroid_shp

## Run Script

In [9]:
def calculate_total_centroids():
    total_centroids = 0
    for _, _ in find_centroid(centroid_folder):
        total_centroids+=1
    print 'total_centroids: ' + str(total_centroids)
    return total_centroids
total_centroids = calculate_total_centroids()

total_centroids: 5


In [ ]:
count = 0
for centroid_directory, centroid_shp in find_centroid(centroid_folder):
    index_setup(network_dataset_shp, centroid_shp)
    inOrigins, inDestinations, inNetworkDataset, outGeodatabase = setup_od_inputs(centroid_shp, centroid_directory, network_dataset_nd)
    generate_od_matrix(inOrigins, inDestinations, inNetworkDataset, outGeodatabase) # 16 minutes
    output_csv(centroid_directory, outGeodatabase) # 2 minutes
    count+=1
    print "completed" + str(count) + "out of " + str(total_centroids)

In [ ]:
for centroid_directory, centroid_shp in find_centroid(centroid_folder):
    print centroid_shp
    inTable = arcpy.SearchCursor(centroid_shp)
    outLocation = centroid_directory
    outTable = "centroid.csv"
    arcpy.TableToTable_conversion(inTable, outLocation, outTable)
    break

## Make New OD Matrix With Origin and Destination Census IDs

In [23]:
def find_centroids_with_od_output(centroid_folder):
    for directory, _, files in os.walk(os.path.join(env.workspace, centroid_folder)):
        for file in files:
            if "od_output.csv" in files:
                if ".shp" == file[-4:]:
                    centroid_directory = directory
                    centroid_shp = os.path.join(directory, file)
                    yield centroid_directory, centroid_shp
def count_total():
    count = 0
    for each,_ in find_centroids_with_od_output(centroid_folder):
        count += 1
    return count

def add_census_id_to_od_matrix(centroid_folder):
    total = str(count_total())
    print("There are " + total + " matrices to update")
    count = 0
    for centroid_directory, centroid_shp in find_centroids_with_od_output(centroid_folder):
        output_df = pd.read_csv(os.path.join(centroid_directory, "od_output.csv"))
        rows=arcpy.SearchCursor(centroid_shp)
        ids = []
        #retrieve all the census IDs
        for row in rows:
            OID = row.getValue("FID")
            census_id=row.getValue("geo2010")
            ids.append([OID, census_id])
        centroid_df =  pd.DataFrame(data=ids, columns = ["OID","census_id"])
        #create origin and destination matrix to merge with output
        origins = centroid_df.rename(columns={"census_id":"census_origin", "OID":"OriginOID"}).astype("int64")
        destinations = centroid_df.rename(columns={"census_id":"census_destination", "OID":"DestinationOID"}).astype("int64")
        #merge origin and destination census_ids with OD_matrix
        result = output_df.merge(origins, how="left").merge(destinations, how="left")
        #store new csv
        output_file = os.path.join(centroid_directory, "OD_matrix.csv")
        output_columns = ["Total_Time", "Total_Distance", "OriginOID", "DestinationOID", "census_origin", "census_destination"]
        result[output_columns].to_csv(output_file)
        count+= 1
        print(str(count) + "/" + total + " complete")

In [12]:
add_census_id_to_od_matrix(centroid_folder)